In [41]:
#####Latent Dirichlet Allocation Model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import gensim
from scipy.special import gammaln
from scipy.misc import factorial
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns
import time

####データの発生####
##データの設定
k = 15   #トピック数
d = 3000   #文書数
v  = 1000   #語彙数
w = poisson(150, d)   #1文書あたりの単語数
f = sum(w)   #総単語数

In [42]:
##パラメータの設定
#ディレクリ分布のパラメータの設定
alpha0 = np.repeat(0.2, k)   #文書のディレリ事前分布のパラメータ
alpha1 = np.repeat(0.15, v)   #単語のディレクリ事前分布のパラメータ

#ディレクリ分布からパラメータを発生
thetat = theta = np.random.dirichlet(alpha0, d)   #文書のトピック分布をディレクリ乱数から発生
phit = phi = np.random.dirichlet(alpha1, k)    #単語のトピック分布をデレクレリ乱数から発生

In [43]:
##多項分布より単語頻度行列を生成
WX = np.zeros((d, v), dtype='int')
Z = [i for i in range(d)]
vec = np.arange(0, k)

for i in range(d):
    #文書のトピックを生成
    z = multinomial(1, theta[i, :], w[i])   #文書のトピック分布を発生
    z_vec = np.dot(z, vec)

    #トピック割当から単語を生成
    word = np.zeros((w[i], v))
    for j in range(w[i]):
        word[j, :] = multinomial(1, phi[z_vec[j], :], 1)

    WX[i, :] = np.sum(word, axis=0)
    Z[i] = z_vec

In [44]:
####トピックモデルのためのデータと関数の準備####
##それぞれの文書中の単語の出現をベクトルに並べる
##データ推定用IDを作成
ID_list = [i for i in range(d)]
wd_list = [i for i in range(d)]

#文書ごとに求人IDおよび単語IDを作成
for i in range(d):
    ID_list[i] = np.repeat(i, w[i])
    num1 = (WX[i, :] > 0) * np.arange(1, v+1)
    num2 = num1[num1!=0]
    W1 = WX[i, WX[i, :] > 0]
    number = np.repeat(num2, W1)
    wd_list[i] = number

#リストをベクトル変換
wd = np.zeros(sum(w), dtype='int')
ID_d = np.zeros(sum(w), dtype='int')
start = 0

for i in range(d):
    wd[start:start+w[i]] = wd_list[i] - 1
    ID_d[start:start+w[i]] = ID_list[i]
    start += w[i] 

In [45]:
##インデックスを作成
doc_list = [i for i in range(d)]
doc_vec = [i for i in range(d)]
word_list = [i for i in range(v)]
word_vec = [i for i in range(v)]
index = np.array(range(f))

for i in range(d):
    doc_list[i] = index[ID_d==i]
    doc_vec[i] = np.repeat(1, doc_list[i].shape)
for i in range(v):
    word_list[i] = index[wd==i]
    word_vec[i] = np.repeat(1, word_list[i].shape)

In [46]:
####マルコフ連鎖モンテカルロ法でLDAのパラメータをサンプリング####
##単語ごとに尤度と負担率を計算する関数
def burden_fr(f, theta, phi, wd, w, k):
    Bur = np.zeros((f, k))
    Bur0 = np.zeros((f, k))

    #負担係数を計算
    for j in range(k):
        Bi = np.repeat(theta[:, j], w) * phi[j, wd]
        Bur[:, j] = Bi

    #負担率の分母部分
    Bur_sums = np.sum(Bur, axis=1)
    for l in range(k):
        Bur0[:, l] = Bur_sums 

    #負担率と混合率を計算
    Br = Bur / Bur0   #負担率の計算
    r = np.sum(Br, axis=0) / np.sum(Br)   #混合率の計算
    return Br, Bur, r

In [47]:
##アルゴリズムの設定
R = 5000
keep = 2
burnin = 500
iter = 0 
disp =10

In [48]:
##事前分布の設定
alpha1 = 1.0 
beta1 = 0.5

In [49]:
##パラメータの初期値
mu = np.sum(WX, axis=0)/np.sum(WX)
phi = np.random.dirichlet(np.repeat(1, v), k)
theta = np.random.dirichlet(np.repeat(1, k), d)

In [50]:
##パラメータの保存用配列
THETA = np.zeros((d, k, R//keep), dtype='int')
PHI = np.zeros((k, v, R//keep), dtype='int')
SEG = np.zeros((f, k), dtype='int')

In [51]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    ##単語トピックを生成
    #潜在確率を計算
    out = burden_fr(f, theta, phi, wd, w, k)
    Br = out[0]
    r = out[2]

    #多項分布よりトピックを生成
    Zi = np.array([np.random.multinomial(1, Br[i, :], 1) for i in range(f)]).reshape((f, k))
    zi_vec = np.dot(Zi, np.arange(1, k+1))
    Zi_T = Zi.T

    ##パラメータを更新
    #トピック分布thetaをサンプリング
    wsum = np.zeros((d, k))
    for i in range(d):
        wsum[i :] = np.dot(Zi_T[:, doc_list[i]], doc_vec[i]) + alpha1   #ディクレリ分布のパラメータ
        theta[i, :] = np.random.dirichlet(wsum[i, :], 1)

    #単語分布phiをサンプリング
    vsum = np.zeros((k, v))
    for j in range(v):
        vsum[:, j] = np.dot(Zi_T[:, word_list[j]], word_vec[j]) + beta1   #ディクレリ分布のパラメータ
    phi = np.array([np.random.dirichlet(vsum[i, :], 1) for i in range(k)]).reshape((k, v))
        
    ##サンプリング結果の格納とパラメータの表示
    #サンプリング結果の保存
    if rp%keep==0:
        mkeep = rp//keep
        THETA[:, :, mkeep] = theta
        PHI[:, :, mkeep] = phi
        
        #バーンインを超えたらトピックを格納
        if rp >= burnin:
            SEG += Zi  
    
        #パラメータの表示
        if rp%disp==0:
            LL = np.sum(np.log(np.sum(out[1], axis=1)))   #対数尤度を計算
            print(rp)
            print(LL)

0
-3130684.958495378
10
-3002904.624209975
20
-2941272.05085049
30
-2879959.193621996
40
-2861282.137684374
50
-2851650.978165074
60
-2848500.6704861494
70
-2844692.648576236
80
-2838427.189436499
90
-2837934.630898177
100
-2837660.6314784684
110
-2837515.249631107
120
-2837439.270155351
130
-2837265.8448201832
140
-2837782.8602804495
150
-2837509.772496189
160
-2837426.578841464
170
-2837550.521929357
180
-2837424.1012327494
190
-2837779.6119840154
200
-2837363.3290966996
210
-2837435.581043686
220
-2837173.3513664817
230
-2837617.5000099894
240
-2837636.3098315047
250
-2837447.6321882205
260
-2837430.900054523
270
-2837464.573548879
280
-2837367.40284317
290
-2837516.767446105
300
-2837709.0770900487
310
-2837708.813806731
320
-2837524.1178090754
330
-2837827.412978327
340
-2837573.1228738693
350
-2837508.269338872
360
-2837372.553364713
370
-2837330.8888130286
380
-2837735.621522203
390
-2837808.1196487243
400
-2837440.1377938227
410
-2837625.7625163943
420
-2837413.4703230564
430
-

3400
-2837445.129061914
3410
-2837457.533684921
3420
-2837747.0256098346
3430
-2837499.595907817
3440
-2837317.1000366067
3450
-2837280.8728293525
3460
-2837378.7513794
3470
-2837327.5197079554
3480
-2837376.8241868475
3490
-2837593.031049363
3500
-2837325.34099627
3510
-2837721.40866859
3520
-2837306.284886029
3530
-2837487.0793843796
3540
-2837524.966615788
3550
-2837205.2025717027
3560
-2837634.711758555
3570
-2837853.171127448
3580
-2837385.835085039
3590
-2837713.582477677
3600
-2837397.6812209915
3610
-2837591.3222735687
3620
-2837393.9057396674
3630
-2837371.9480614285
3640
-2837018.714521746
3650
-2837641.5503979875
3660
-2837570.4029066027
3670
-2837415.744126352
3680
-2837506.6937634554
3690
-2837628.658324048
3700
-2837558.005224152
3710
-2837510.051836365
3720
-2837734.8793239244
3730
-2837562.203747889
3740
-2837729.522783109
3750
-2837621.900509896
3760
-2837329.2123573646
3770
-2837433.222122652
3780
-2837267.6672635945
3790
-2837487.186004398
3800
-2837798.910089517
381

In [ ]:
phi[i, :] = np.random.dirichlet(vsum[i, :], 1)

(15, 500)